In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from data import *
from plotting import *
from regression import *
from utils import *

In [120]:
results_dir = 'results/updated-schema/'
os.makedirs(results_dir, exist_ok=True)

# Data

In [121]:
# Load data
pcd_df = load_pcd_df()

In [122]:
pcd_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Field 57,Organization categorization (from Organization)
0,Llama 3,NaN,NaN,NaN,NaN,NaN,NaN,https://engineering.fb.com/2024/03/12/data-cen...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DBRX,Language,"Chat,Code generation",Mosaic Research Team,NaN,NaN,Open access (restricted use),https://www.databricks.com/blog/introducing-db...,NaN,Introducing DBRX: A New State-of-the-Art Open LLM,...,Industry,NaN,NaN,NaN,NaN,Unreleased,Unreleased,license: https://www.databricks.com/legal/open...,NaN,Industry
2,Claude 3 Haiku,"Multimodal,Language,Vision",Chat,NaN,NaN,NaN,API access,https://www-cdn.anthropic.com/de8ba9b01c9ab7cb...,NaN,"The Claude 3 Model Family: Opus, Sonnet, Haiku",...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry
3,Claude 3 Sonnet,"Multimodal,Language,Vision",Chat,NaN,NaN,NaN,API access,https://www-cdn.anthropic.com/de8ba9b01c9ab7cb...,NaN,"The Claude 3 Model Family: Opus, Sonnet, Haiku",...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry
4,Claude 3 Opus,"Multimodal,Language,Vision",Chat,NaN,SOTA improvement,NaN,API access,https://www-cdn.anthropic.com/de8ba9b01c9ab7cb...,NaN,"The Claude 3 Model Family: Opus, Sonnet, Haiku",...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,Self Organizing System,Other,Pattern recognition,W. A. Clark and B. G. Farley,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455309,93.0,Generalization of pattern recognition in a sel...,...,Academia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Academia
1409,Genetic algorithm,Other,NaN,NA Barricelli,Historical significance,Possibly first computer simulation of a geneti...,NaN,https://link.springer.com/article/10.1007/BF01...,266.0,Numerical testing of evolution theories,...,Academia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Academia
1410,SNARC,Robotics,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,Academia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Academia
1411,Theseus,Robotics,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry


In [123]:
pcd_df.loc[pcd_df['System'] == 'Megatron-BERT']['Model accessibility']

809    Unreleased
Name: Model accessibility, dtype: object

In [124]:
access_df_with_deprecated = pcd_df.dropna(subset=['Publication date', 'Training compute (FLOP)', 'Model accessibility'])
len(access_df_with_deprecated)

239

In [125]:
access_df_with_deprecated['Model accessibility'].unique()

array(['Open access (restricted use)', 'API access', 'Open source',
       'Open access (non-commercial)', 'Hosted access (no API)',
       'Permissive license (depr.)', 'Fully open-source (depr.)',
       'Unreleased', 'Weights available (depr.)'], dtype=object)

In [126]:
for cat in access_df_with_deprecated['Model accessibility'].unique():
    print(cat, len(access_df_with_deprecated.loc[access_df_with_deprecated['Model accessibility'] == cat]))

Open access (restricted use) 17
API access 15
Open source 55
Open access (non-commercial) 13
Hosted access (no API) 4
Permissive license (depr.) 39
Fully open-source (depr.) 52
Unreleased 40
Weights available (depr.) 4


In [127]:
open_access_categories = ['Open source', 'Open access (restricted use)', 'Open access (non-commercial)']
closed_access_categories = ['API access', 'Hosted access (no API)', 'Unreleased']

In [128]:
access_df = access_df_with_deprecated.loc[
    access_df_with_deprecated['Model accessibility'].isin(open_access_categories + closed_access_categories)
]

In [129]:
open_access_df = access_df.loc[access_df['Model accessibility'].isin(open_access_categories)]
len(open_access_df)

85

In [130]:
closed_access_df = access_df.loc[access_df['Model accessibility'].isin(closed_access_categories)]
len(closed_access_df)

59

In [131]:
# Add column with binary access label
access_df['Model open/closed'] = access_df['Model accessibility'].apply(
    lambda x: 'Open' if x in open_access_categories else 'Closed'
)

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_10875/148813227.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Compute percentile filtering

In [132]:
outlier_window_size = 2  # years
start_large_scale_era = '2015-09-01'

In [133]:
pcd_df['Publication date'] = pd.to_datetime(pcd_df['Publication date'])
pcd_df.sort_values('Publication date', inplace=True)
pcd_df.dropna(subset=['Publication date', 'Notability criteria', 'Training compute (FLOP)'], inplace=True)

In [134]:
systems_by_percentile = {}
percentile_interval = 5
for percentile in range(95, -5, -percentile_interval):
  print(percentile)
  percentile_compute_low = np.zeros(len(pcd_df))
  percentile_compute_high = np.zeros(len(pcd_df))
  # Iterate through each row and calculate the 2-year moving average for each date
  for i, (index, row) in enumerate(pcd_df.iterrows()):
    # Define the 2-year window
    start_date = row['Publication date'] - pd.DateOffset(years=outlier_window_size/2)
    end_date = row['Publication date'] + pd.DateOffset(years=outlier_window_size/2)

    # Filter the DataFrame for this window
    window_df = pcd_df[(pcd_df['Publication date'] >= start_date) & (pcd_df['Publication date'] <= end_date)]

    percentile_compute_low[i] = np.percentile(window_df['Training compute (FLOP)'], percentile)
    percentile_compute_high[i] = np.percentile(window_df['Training compute (FLOP)'], percentile + percentile_interval)

  systems_flag = pcd_df['Training compute (FLOP)'] > np.array(percentile_compute_low)
  extra_systems_flag = pcd_df['Training compute (FLOP)'] <= np.array(percentile_compute_high)

  # raise Exception("Edit the following line if you want to consider models released after 2023-12-31.")
  extra_systems = pcd_df['System'][systems_flag & extra_systems_flag & (pcd_df['Publication date'] > pd.to_datetime('2015-09-30'))].values

  systems_by_percentile[percentile] = list(extra_systems)

95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10
5
0


In [135]:
systems_by_percentile

{95: ['GNMT',
  'AlphaGo Master',
  'AlphaGo Zero',
  'AlphaZero',
  'ResNeXt-101 32x48d',
  'Megatron-BERT',
  'OpenAI Five',
  'Meena',
  'GPT-3 175B (davinci)',
  'Megatron-Turing NLG 530B',
  'ERNIE 3.0 Titan',
  'PaLM (540B)',
  'Minerva (540B)',
  'GPT-4',
  'PaLM 2',
  'Inflection-2',
  'Gemini Ultra'],
 90: ['NASv3 (CIFAR-10)',
  'FTW',
  'T5-11B',
  'AlphaStar',
  'mT5-XXL',
  'Switch',
  'Gopher (280B)',
  'Chinchilla',
  'U-PaLM (540B)',
  'GPT-3.5 (text-davinci-003)',
  'Claude 2',
  'Falcon 180B'],
 85: ['AlphaGo Fan',
  'AlphaGo Lee',
  'BigGAN-deep 512x512',
  'Megatron-LM (8.3B)',
  'OpenAI Five Rerun',
  'Turing-NLG',
  'Yuan 1.0',
  'GLaM',
  'LaMDA',
  'OPT-175B',
  'BLOOM-176B',
  'Llama 2-70B',
  'ChatGLM3',
  'Qwen-72B'],
 80: ['JFT',
  'OpenAI TI7 DOTA 1v1',
  'AmoebaNet-A (F=448)',
  'GPT-2 (1.5B)',
  'iGPT-XL',
  'DALL-E',
  'Meta Pseudo Labels',
  'ProtT5-XXL',
  'ByT5-XXL',
  'GOAT',
  'HyperCLOVA',
  'AlphaCode',
  'Flamingo',
  'Parti',
  'BlenderBot 3',
  

In [136]:
compute_percentile_threshold = 50
selected_systems = []
for percentile in range(compute_percentile_threshold, 100, 5):
    selected_systems.extend(systems_by_percentile[percentile])
selected_systems

['BIDAF',
 'Transformer',
 'GPT',
 'GBERT-Large',
 'wave2vec 2.0 LARGE',
 'AlphaFold 2',
 'DeBERTa',
 'HuBERT',
 'XGLM',
 'Imagen',
 'NLLB',
 'ESM2-3B',
 'LLaMA-7B',
 'LLaMA-13B',
 'WizardLM-7B',
 'Pangu-Weather',
 'LSTM (Hebbian, Cache, MbPA)',
 'SciBERT',
 'DD-PPO',
 'ViT-Huge/14',
 'MSA Transformer',
 'M6-T',
 'T0-XXL',
 'Whisper',
 'XGen-7B',
 'DeepStack',
 'PNASNet-5',
 'Population-based DRL',
 'Mesh-TensorFlow Transformer 2.9B (translation)',
 'Megatron-LM (355M)',
 'T5-3B',
 'CamemBERT',
 'Noisy Student (L2)',
 'Once for All',
 'CLIP (ViT L/14@336px)',
 'ERNIE 3.0',
 'BASIC-L',
 'XGLM-7.5B',
 'ESM2-15B',
 'PaLI',
 'Taiyi-Stable Diffusion',
 'Llama 2-7B',
 'Nemotron-3-8B',
 'ResNet-152 (ImageNet)',
 'PolyNet',
 'MoE',
 'YOLOv3',
 'Transformer (Adaptive Input Embeddings)',
 'Mesh-TensorFlow Transformer 4.9B (language modelling)',
 'BERT-Large-CAS (PTB+WT2+WT103)',
 'Conformer + Wav2vec 2.0 + Noisy Student',
 'ProtT5-XXL-BFD',
 'CogView',
 'ALIGN',
 'Florence',
 'Stable Diffusion (

# Regression

In [137]:
access_df['Publication date (float)'] = datetime_to_float_year(
    pd.to_datetime(access_df['Publication date'])
)

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_10875/3991044387.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [138]:
filtered_access_df = access_df.loc[access_df['System'].isin(selected_systems)]

In [139]:
reg_results = fit_ols_regression(
    filtered_access_df,
    ['Publication date (float)'],
    'Training compute (FLOP)',
    logy=True
)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     90.94
Date:                Mon, 01 Apr 2024   Prob (F-statistic):           4.23e-14
Time:                        11:46:24   Log-Likelihood:                -81.836
No. Observations:                  69   AIC:                             167.7
Df Residuals:                      67   BIC:                             172.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1104.7906    118.274     -9.341      0.000   -1340.867    -868.715
x1             0.5578      0.058      9.536      0.000       0.441       0.675
==============================================================================
Omnibus:                        0.956   Durbin-Watson:                   1.673
Prob(Omnibus):                  0.620   Jarque-Bera (JB):                0.889
Skew:                          -0.001   Prob(JB):                        0.641
Kurtosis:                       2.444   Cond. No.                     2.47e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.47e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [140]:
open_reg_results = fit_ols_regression(
    filtered_access_df[filtered_access_df['Model open/closed'] == 'Open'],
    ['Publication date (float)'],
    'Training compute (FLOP)',
    logy=True
)
open_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     68.70
Date:                Mon, 01 Apr 2024   Prob (F-statistic):           2.99e-09
Time:                        11:46:25   Log-Likelihood:                -26.927
No. Observations:                  32   AIC:                             57.85
Df Residuals:                      30   BIC:                             60.79
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1175.2612    144.549     -8.131      0.000   -1470.470    -880.052
x1             0.5924      0.071      8.288      0.000       0.446       0.738
==============================================================================
Omnibus:                        0.248   Durbin-Watson:                   1.572
Prob(Omnibus):                  0.883   Jarque-Bera (JB):                0.443
Skew:                          -0.013   Prob(JB):                        0.801
Kurtosis:                       2.424   Cond. No.                     2.85e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.85e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [141]:
closed_reg_results = fit_ols_regression(
    filtered_access_df[filtered_access_df['Model open/closed'] == 'Closed'],
    ['Publication date (float)'],
    'Training compute (FLOP)',
    logy=True
)
closed_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     73.08
Date:                Mon, 01 Apr 2024   Prob (F-statistic):           4.33e-10
Time:                        11:46:25   Log-Likelihood:                -41.676
No. Observations:                  37   AIC:                             87.35
Df Residuals:                      35   BIC:                             90.57
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1195.4759    142.575     -8.385      0.000   -1484.917    -906.034
x1             0.6029      0.071      8.549      0.000       0.460       0.746
==============================================================================
Omnibus:                        1.533   Durbin-Watson:                   1.710
Prob(Omnibus):                  0.465   Jarque-Bera (JB):                1.155
Skew:                          -0.430   Prob(JB):                        0.561
Kurtosis:                       2.907   Cond. No.                     2.28e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.28e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [142]:
print('All:')
print_growth_rates(reg_results)
print('Open:')
print_growth_rates(open_reg_results)
print('Closed:')
print_growth_rates(closed_reg_results)

All:
Adj. R^2=0.57
0.56 OOMs/year (95% CI: 0.44, 0.67)
3.6x/year (95% CI: 2.8x, 4.7x)
doubling time of 6 months (95% CI: 5, 8)
Open:
Adj. R^2=0.69
0.59 OOMs/year (95% CI: 0.45, 0.74)
3.9x/year (95% CI: 2.8x, 5.5x)
doubling time of 6 months (95% CI: 5, 8)
Closed:
Adj. R^2=0.67
0.60 OOMs/year (95% CI: 0.46, 0.75)
4.0x/year (95% CI: 2.9x, 5.6x)
doubling time of 6 months (95% CI: 5, 8)


## Predictions

In [143]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [144]:
predicted_open_df = get_predictions(open_reg_results, pred_years, ['Publication date (float)'])
predicted_open_df['Publication date'] = predicted_open_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_open_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,18.490955,0.530568,17.407390,19.574520,16.886008,20.095902,2015.00000,2015-01-01
1,18.550797,0.523486,17.481695,19.619898,16.955579,20.146015,2015.10101,2015-02-06
2,18.610638,0.516408,17.555993,19.665284,17.025072,20.196204,2015.20202,2015-03-15
3,18.670480,0.509333,17.630283,19.710678,17.094487,20.246473,2015.30303,2015-04-21
4,18.730322,0.502263,17.704564,19.756080,17.163822,20.296822,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,24.175916,0.194512,23.778670,24.573163,22.927098,25.424735,2024.59596,2024-08-06
96,24.235758,0.200685,23.825905,24.645611,22.982873,25.488644,2024.69697,2024-09-12
97,24.295600,0.206925,23.873002,24.718198,23.038487,25.552712,2024.79798,2024-10-19
98,24.355442,0.213228,23.919973,24.790911,23.093944,25.616939,2024.89899,2024-11-25


In [145]:
predicted_closed_df = get_predictions(closed_reg_results, pred_years, ['Publication date (float)'])
predicted_closed_df['Publication date'] = predicted_closed_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_closed_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,19.285028,0.493052,18.284080,20.285976,17.433304,21.136753,2015.00000,2015-01-01
1,19.345923,0.486169,18.358948,20.332899,17.501714,21.190132,2015.10101,2015-02-06
2,19.406818,0.479293,18.433801,20.379835,17.570042,21.243594,2015.20202,2015-03-15
3,19.467713,0.472425,18.508639,20.426787,17.638285,21.297141,2015.30303,2015-04-21
4,19.528608,0.465564,18.583462,20.473753,17.706443,21.350773,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,25.070039,0.236523,24.589871,25.550207,23.439841,26.700237,2024.59596,2024-08-06
96,25.130934,0.242578,24.638474,25.623394,23.497073,26.764795,2024.69697,2024-09-12
97,25.191829,0.248690,24.686961,25.696696,23.554185,26.829472,2024.79798,2024-10-19
98,25.252723,0.254854,24.735342,25.770105,23.611179,26.894268,2024.89899,2024-11-25


## Differences between trends

In [146]:
NUM_SAMPLES = 1000

In [147]:
open_reg_results.conf_int()

array([[-1.47047018e+03, -8.80052215e+02],
       [ 4.46454791e-01,  7.38410871e-01]])

In [154]:
# Open
open_ci = open_reg_results.conf_int()
open_slope = lognorm_from_90_ci(
    10**open_ci[1][0],
    10**open_ci[1][1],
    NUM_SAMPLES,
)
open_intercept = normal_from_90_ci(
    open_ci[0][0],
    open_ci[0][1],
    NUM_SAMPLES,
)
# Closed
closed_ci = closed_reg_results.conf_int()
closed_slope = lognorm_from_90_ci(
    10**closed_ci[1][0],
    10**closed_ci[1][1],
    NUM_SAMPLES,
)
closed_intercept = normal_from_90_ci(
    closed_ci[0][0],
    closed_ci[0][1],
    NUM_SAMPLES,
)

In [157]:
print_median_and_ci(open_slope)
print_median_and_ci(open_intercept)
print_median_and_ci(closed_intercept)

Median: 3.9 [90% CI: 2.8, 5.6]
Median: -1.2e+03 [90% CI: -1.5e+03, -8.8e+02]
Median: -1.2e+03 [90% CI: -1.5e+03, -9e+02]


In [89]:
# Calculate when the closed compute trend was equal to the open compute trend today
current_date = 2024.25
current_open_compute = open_reg_results.params[1] * current_date + open_reg_results.params[0]
current_open_compute

23.970958636853084

In [92]:
current_closed_compute = closed_reg_results.params[1] * current_date + closed_reg_results.params[0]
current_closed_compute

24.861474095069525

In [94]:
compute_gap = current_closed_compute - current_open_compute
print(f'Open access models are {compute_gap:.1f} OOMs behind closed access models')

Open access models are 0.9 OOMs behind closed access models


In [90]:
closed_compute_match_date = (current_open_compute - closed_reg_results.params[0]) / closed_reg_results.params[1]
closed_compute_match_date

2022.772846203448

In [95]:
open_time_lag = current_date - closed_compute_match_date
print(f'Open access models are {open_time_lag:.1f} years behind closed access models')

Open access models are 1.5 years behind closed access models


# Plots

In [50]:
fig = px.scatter(
    access_df_with_deprecated,
    x='Publication date',
    y='Training compute (FLOP)',
    color='Model accessibility',
    hover_data=['System'],
    log_y=True,
)

save_plot(fig, results_dir, 'training_compute_by_model_accessibility_depr')

fig.show()

In [51]:
fig = px.scatter(
    access_df,
    x='Publication date',
    y='Training compute (FLOP)',
    color='Model accessibility',
    hover_data=['System'],
    log_y=True,
)

save_plot(fig, results_dir, 'training_compute_by_model_accessibility')

fig.show()

In [65]:
fig = px.scatter(
    # access_df,
    access_df.loc[access_df['System'].isin(selected_systems)],
    x='Publication date',
    y='Training compute (FLOP)',
    color='Model open/closed',
    hover_data=['System'],
    log_y=True,
)

save_plot(fig, results_dir, 'training_compute_by_open_closed')

fig.show()

In [116]:
fig = px.scatter(
    # access_df,
    access_df.loc[access_df['System'].isin(selected_systems)],
    x='Publication date',
    y='Training compute (FLOP)',
    color='Model open/closed',
    hover_data=['System'],
    log_y=True,
)

# Marker color
# fig.update_traces(
#     marker=dict(
#         color='rgb(0,100,200)',
#     ),
#     selector=dict(mode='markers'),
# )

# Shade in CI
fig.add_scatter(
    x=predicted_open_df['Publication date'],
    y=10**predicted_open_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_open_df['Publication date'],
    y=10**predicted_open_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(230,100,0,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_open_df['Publication date'],
    y=10**predicted_open_df['mean'],
    mode='lines',
    line=dict(color='rgb(230,100,0)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.add_scatter(
    x=predicted_closed_df['Publication date'],
    y=10**predicted_closed_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_closed_df['Publication date'],
    y=10**predicted_closed_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_closed_df['Publication date'],
    y=10**predicted_closed_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
# fig.update_xaxes(title_text='Publication date')
# fig.update_yaxes(title_text='Cost (2023 USD)')

# title


# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'open_closed_regression')

fig.show()

In [164]:
access_df.columns

Index(['System', 'Domain', 'Task', 'Authors', 'Notability criteria',
       'Notability criteria notes', 'Model accessibility', 'Link', 'Citations',
       'Reference', 'Publication date', 'Organization', 'Parameters',
       'Parameters notes', 'Training compute (FLOP)', 'Training compute notes',
       'Training dataset', 'Training dataset notes',
       'Training dataset size (datapoints)', 'Dataset size notes', 'Epochs',
       'Inference compute (FLOP)', 'Inference compute notes',
       'Training time (hours)', 'Training time notes', 'Training hardware',
       'Approach', 'Training compute cost (2020 USD)', 'Compute cost notes',
       'Compute sponsor categorization', 'Confidence', 'Abstract',
       'Last modified', 'Created By', 'Benchmark data', 'Exclude',
       'Country (from Organization)', 'Base model', 'Finetune compute (FLOP)',
       'Finetune compute notes', 'Hardware quantity', 'Hardware utilization',
       'Training cost trends', 'Training cloud compute vendor',
 